## BER energy data

Data taken from the public BER dataset within this notebook. 
Aim is to understand build a structure that can be used to perform analysis on future datasets that contain a smaller number of the features. Creating a robust target variable ("BER") using the fewest number of available features is the long term goal. To align this strategy with the data that a company will have available will be the challenge of understanding how the features are collected. Will features within this dataset perform better if they are before or after the BER assessment to provide detailed information on what drives the BER.

Data Source: public search data [website](https://ndber.seai.ie/BERResearchTool/ber/search.aspx). 

- Build a baseline ML classification model e.g., boosting tree, help understand important features
- Perform unsupervised learning to review clusters of variables that have similar characteristics
- Review which variables could be transformed and/or combined to benefit model accuracy
- Are there any features that could be collected in external datasets that are similar to data shown

In [1]:
# Training examples using Jupyter Notebook
# Aim is to understand example code that can be moved to GitHub for future use

# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sys
import installPack # took the original code that was being used to install a new package and wrapped it in a py script
import polars as pl
import plotly.express as px

In [2]:
# List of libraries to import
requirements = ["pyarrow"]
for requirement in requirements:
    installPack.installPackage(requirement)

In [3]:
import pyarrow as pa
import tidypolars as tp
from tidypolars import col, desc

In [4]:
# text file to scan
txt_file = "BERPublicsearch.txt"

pl_lazy = pl.scan_csv(txt_file, separator="\t").fetch(100)

In [5]:
pl_lazy

CountyName,DwellingTypeDescr,Year_of_Construction,TypeofRating,EnergyRating,BerRating,GroundFloorArea(sq m),UValueWall,UValueRoof,UValueFloor,UValueWindow,UvalueDoor,WallArea,RoofArea,FloorArea,WindowArea,DoorArea,NoStoreys,CO2Rating,MainSpaceHeatingFuel,MainWaterHeatingFuel,HSMainSystemEfficiency,MultiDwellingMPRN,TGDLEdition,MPCDERValue,HSEffAdjFactor,HSSupplHeatFraction,HSSupplSystemEff,WHMainSystemEff,WHEffAdjFactor,SupplSHFuel,SupplWHFuel,SHRenewableResources,WHRenewableResources,NoOfChimneys,NoOfOpenFlues,NoOfFansAndVents,…,SecondHeatGenPlantEff,SecondPercentageHeat,ThirdBoilerFuelType,ThirdHeatGenPlantEff,ThirdPercentageHeat,SolarSpaceHeatingSystem,TotalPrimaryEnergyFact,TotalCO2Emissions,FirstWallType_Description,FirstWallDescription,FirstWallArea,FirstWallUValue,FirstWallIsSemiExposed,FirstWallAgeBandId,FirstWallTypeId,SecondWallType_Description,SecondWallDescription,SecondWallArea,SecondWallUValue,SecondWallIsSemiExposed,SecondWallAgeBandId,SecondWallTypeId,ThirdWallType_Description,ThirdWallDescription,ThirdWallArea,ThirdWallUValue,ThirdWallIsSemiExposed,ThirdWallAgeBandId,ThirdWallTypeId,SA_Code,prob_smarea_error_0corr,prob_smarea_error_100corr,RER,RenewEPnren,RenewEPren,CPC,EPC
str,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,str,str,f64,str,i64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,f64,f64,str,str,f64,f64,str,i64,i64,str,str,f64,f64,str,i64,i64,str,str,f64,f64,str,i64,i64,str,f64,f64,str,str,str,f64,f64
"""Co. Waterford""","""Mid-terrace ho…",2008,"""Final …","""B3""",129.9,108.0,0.27,0.16,0.23,1.7,3.0,49.1,45.98,40.5,9.82,3.78,3,33.01,"""Heating Oil …","""Heating Oil …",85.5,"""NO""",1,34.1,1.0,0.1,30.0,85.5,1.0,15,1,2,2,1,0,8,…,null,null,null,null,null,null,null,null,"""300mm Cavity""","""300mm ins cav""",35.6,0.29,"""No""",null,4,"""Other""","""room in roof""",13.5,0.22,"""No""",null,9,null,null,null,null,null,null,null,"""227033002""",5.5623e-7,0.0025,null,null,null,null,null
"""Co. Galway""","""Semi-detached …",2009,"""Final …","""C1""",165.61,122.78,0.22,0.16,0.19,2.7,2.97,124.45,65.79,65.79,15.7,3.7,2,42.17,"""Heating Oil …","""Heating Oil …",85.7,"""NO""",2,45.47,1.0,0.1,30.0,85.7,1.0,15,28,2,2,1,0,3,…,null,null,null,null,null,null,null,null,"""300mm Filled C…","""Walls - South""",81.75,0.22,"""No""",null,5,"""300mm Filled C…","""Walls - West""",22.35,0.22,"""No""",null,5,"""300mm Filled C…","""Walls - East""",20.35,0.22,"""No""",null,5,"""67170019""",5.5623e-7,0.0025,null,null,null,1.249,1.046
"""Co. Cork""","""Apartment""",2006,"""Existing …","""D2""",260.75,35.73,0.37,0.0,0.34,3.1,3.0,35.29,0.0,35.73,22.64,1.85,1,49.41,null,null,null,"""NO""",0,41.61,null,null,null,null,null,null,null,null,null,0,0,2,…,0,0,1,0,0,0,1.54,0.284,"""300mm Filled C…","""Living""",28.1,0.37,"""No""",10,5,"""300mm Filled C…","""Bedroom""",7.19,0.37,"""No""",10,5,null,null,null,null,null,null,null,"""48052009""",0.000031,0.0025,null,null,null,null,null
"""Co. Dublin""","""Mid-floor apar…",2002,"""Existing …","""B3""",138.07,72.0,0.52,0.0,0.0,3.3,3.0,44.41,0.0,0.0,11.23,1.85,1,28.78,"""Mains Gas …","""Mains Gas …",79.2,"""NO""",0,28.59,1.0,0.1,100.0,79.2,1.0,1,28,2,2,0,0,5,…,null,null,null,null,null,null,null,null,"""300mm Cavity""","""South Elevatio…",20.54,0.55,"""No""",9,4,"""225mm Solid br…","""West Elevation…",12.23,0.45,"""Yes""",9,2,"""300mm Cavity""","""East Elevation…",11.64,0.55,"""No""",9,4,"""267112011""",0.000002,0.0025,null,null,null,null,null
"""Co. Dublin""","""Mid-floor apar…",2002,"""Existing …","""B3""",145.9,85.84,0.52,0.0,0.0,3.3,3.0,57.43,0.0,0.0,11.23,1.85,1,30.07,"""Mains Gas …","""Mains Gas …",79.2,"""NO""",0,27.12,1.0,0.1,0.0,79.2,1.0,1,28,2,2,0,0,5,…,null,null,null,null,null,null,null,null,"""300mm Cavity""","""North Elevatio…",21.9,0.55,"""No""",9,4,"""225mm Solid br…","""West Elevation…",17.35,0.45,"""Yes""",9,2,"""300mm Cavity""","""East Elevation…",13.1,0.55,"""No""",9,4,"""267112011""",0.000002,0.0025,null,null,null,null,null
"""Du

In [ ]:
# Import the entire text file
pl_df = pl.read_csv(txt_file, separator="\t", ignore_errors=True)

In [ ]:
# Shape of the file
pl_df.shape

In [ ]:
pl_df.estimated_size("gb")

Polars import shows that there was 63,093 extra rows when importing the text file compared to conversion of file to csv format and then importing. Extra data within excel could not be processed correctly.

In [ ]:
type(pl_df)

In [ ]:
# Renaming variables
pl_df = pl_df.rename(
    {
        "CountyName" : "County"
        ,"DwellingTypeDescr" : "DwellingType"
    }
)

In [ ]:
pl_df.head(5)

In [ ]:
# column names
pl_df.columns
# pl_df.describe()

In [ ]:
# Home Energy Scheme Ireland upgrades - Binary flag
# Is there a way to connect the before and after to understand the improvement?
pl_df.groupby(pl.col('HESSchemeUpgrade')).agg(pl.count('EnergyRating')).sort("EnergyRating", descending=True)

In [ ]:
# Aggregations and grouping
table1 = (
    pl_df
    .groupby(pl.col('HESSchemeUpgrade'))
    .agg(pl.count('BerRating'))
    .with_columns((pl.col("BerRating") / pl.sum("BerRating")).alias("percent_count"))
    # .sort("BerRating", descending=True) # Sort descending by volume of BerRating
    # .sort("EnergyRating")
)

table1

In [ ]:
# Aggregations and grouping
pl_df.groupby(pl.col('Year_of_Construction')).agg(pl.count('EnergyRating')).sort("EnergyRating", descending=True)

In [ ]:
# Aggregations and grouping
pl_df.groupby(pl.col('Year_of_Construction')).agg(pl.count('EnergyRating')).sort("EnergyRating", descending=True)

In [ ]:
# Aggregations and grouping
table1 = (
    pl_df
    .groupby(pl.col('EnergyRating'))
    .agg(pl.count('BerRating'))
    .with_columns((pl.col("BerRating") / pl.sum("BerRating")).alias("percent_count"))
    # .sort("BerRating", descending=True) # Sort descending by volume of BerRating
    .sort("EnergyRating")
)

table1

In [ ]:
# Run pivot table
pl_df1 = pl_df.pivot(
    values = "CO2Rating"
    ,index = "County"
    ,columns = "DwellingType"
    ,aggregate_function="count" # options: ["count", "mean", "sum"]
)
pl_df1

In [ ]:
# Aggregations and grouping
table1 = (
    pl_df
    .groupby(pl.col('HESSchemeUpgrade', 'EnergyRating'))
    .agg(pl.count('BerRating'))
    .with_columns((pl.col("BerRating") / pl.sum("BerRating")).alias("percent_count"))
    # .sort("BerRating", descending=True) # Sort descending by volume of BerRating
    .sort("EnergyRating")
)

table1

In [ ]:
# Display the type of variable
type(table1)

In [ ]:
# Histogram - in order to use the plotly express method the KW parameters (x, y) have to be converted into pandas. Conversion is possible using the to_series() method
fig = px.histogram(table1 
                   ,x=table1.select("EnergyRating").to_series()
                   ,y=table1.select("BerRating").to_series()
                  )
fig.show()

In [ ]:
# Histogram alternative
# Can convert the polars dataframe to pandas. Requires pyarrow library to use method to_pandas()
# fig1 = px.histogram(
#     table1.to_pandas()
#     ,x = "EnergyRating"
#     ,y = "BerRating"
# )
# fig1.show()

In [ ]:
# View of the small areas
# General MI. Could find the location mappings and apply to a map to re-create the BER Map
# How to map the SA_Code to text (does a reference table exist??)
table2 = (
    pl_df
    .groupby(pl.col('SA_Code'))
    .agg(pl.count('BerRating'),
         pl.median('BerRating').alias('median_ber'))
    .with_columns((pl.col("BerRating") / pl.sum("BerRating")).alias("percent_count"))
    .sort("BerRating", descending=True) # Sort descending by volume of BerRating
    # .sort("BerRating")
)

table2

In [ ]:
# Test for the SA_code that matches area BALLYGALL B
table2.filter(pl.col("SA_Code") == "268013003")

## 1. Feature engineering

In [ ]:
# Mapping energy rating
# er_curr = list[(pl_df['EnergyRating']).unique()]
# er_curr
# Creating the dictionary to aid with mapping the initial variable displaying the 15 grade values
er_curr = ["A1", "A2", "A3", "B1", "B2", "B3", "C1", "C2", "C3", "D1", "D2", "E1", "E2", "F ", "G "]
er_fin = ["A", "A", "A", "B", "B", "B", "C", "C", "C", "D", "D", "E", "E", "F", "G"]
# Connects the two lists into a dictionary
er_dict = dict(zip(er_curr, er_fin))

In [ ]:
# show dictionary that was created
er_dict

In [ ]:
# Map new set of values. The logic creates the new variable with the chaining method. A method called alias is used to assign the new feature name
pl_df = (pl_df
 .with_columns(pl.col("EnergyRating").map_dict(er_dict).alias("ER1"))
 # .select(pl.col(["EnergyRating", "ER1"]))
)


In [ ]:
pl_df.head()

In [ ]:
# Work to cast the string format to categorical
pl_df = (
    pl_df.with_columns(
        pl.col('County').cast(pl.Categorical),
        pl.col('DwellingType').cast(pl.Categorical),
        pl.col('MainSpaceHeatingFuel').cast(pl.Categorical),
        pl.col('MainWaterHeatingFuel').cast(pl.Categorical)
    )
)

In [ ]:
pl_df.head()

In [ ]:
# Display the list of columns within the dataframe
pl_df.columns

In [ ]:
# Confirm that the mapping dictionary has been applied correctly with the count aggregation
(pl_df
 .groupby(["EnergyRating", "ER1"]).agg(pl.count("County"))
)

In [ ]:
# Understanding datatype of variables within polars dataframe
schema_dict = pl_df.schema
# schema_dict # displays each feature name with the data type within a dictionary format

In [ ]:
# Display the values associated to each dictionary key. Will show the data type 
schema_vals = schema_dict.values()
schema_vals

In [ ]:
# Counter can be used as a method to perform a aggregation of a dictionary. When applied it will produce a descending order output showing feature value with highest count. Great alternative to value_counts() method
from collections import Counter
count = Counter(schema_vals)
print(count)

## 2. Build ML model

In [ ]:
pip install catboost

In [ ]:
# Working with xgboost multi-class classification model
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Target = ER1
y_pl = (pl_df
     .select(["ER1"])
).to_series()

X_orig = (pl_df
     .select(pl.exclude(["ER1", "EnergyRating"]))
)

# Take a selection of the features to produce a baseline model. Initial model run attempted to use all feature variables (excluding target), however this produced an error. 
# Next step would be to produce a data pipeline to understand what adjustments would be required to check if the features causing current issues are observed to be significant.
X_pl = (pl_df
    .select([
        'GroundFloorArea(sq m)',
        'Year_of_Construction',
        'DwellingType',
        'County',
        'MainSpaceHeatingFuel',
        'MainWaterHeatingFuel',
        'UValueWall',
        'UValueRoof',
        'UValueFloor',
        'UValueWindow',
        'UvalueDoor',
        'WallArea',
        'RoofArea',
        'FloorArea',
        'WindowArea',
        'DoorArea',
        'NoStoreys'])
)

In [ ]:
X_test = X_pl.to_arrow().to_pandas()

In [ ]:
dir(X_test)

In [ ]:
# X = X_pl.to_pandas()
y = y_pl.to_series()

In [ ]:
X.head()

In [ ]:
# Need to encode the target to allow the xgboost model to work
lc = LabelEncoder()
lc = lc.fit(y)
lc_y = lc.transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, lc_y, test_size=0.20, random_state=5)

In [ ]:
lc_y

In [ ]:
X_train.columns

In [ ]:
# help(XGBClassifier)

In [ ]:
# Define and fit model
# model = XGBClassifier(enable_categorical=True)
model = CatBoostClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100.0}')

In [ ]:
print(model.feature_importances_)

In [ ]:
# Review key features
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
plt.title('Feature Importance')


Explainable AI

[Article](https://blog.salesforceairesearch.com/omnixai/)
Understand how this could be embedded into the process to aid with model review

Review the tabular classification analysis [link](https://github.com/salesforce/OmniXAI/blob/main/tutorials/tabular_classification.ipynb) 

### 3. Feature selection

In [ ]:
# Review features with low variance
# These are typically features that have low cardinality with large concentration of the sample with a single value
thres = (.8*(1-.8))
thres

In [ ]:
# import library
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# Apply to X_orig - will have to run with only the numeric
# selector = VarianceThreshold()
# selector.fit_transform(X_orig)

In [ ]:
type(X_orig)

In [ ]:
X_orig.select(['BerRating', 'TypeofRating']).head()

In [ ]:
(X_orig
 .to_pandas()
 .nunique()
)

In [ ]:
# Check for cardinality
card_list = (
    X_orig
    .select(pl.all().n_unique())
    .transpose(include_header=True, column_names=['count'])
    # .groupby(pl.col('count'))
    # .agg(pl.count('column'))
    # .sort('count')
    .filter(pl.col('count') <= 5)
    ['column']
    .to_list()
)
# card_list

In [ ]:
card_list

In [ ]:
(X_orig
 .select(card_list)
 .head()
)

In [ ]:
X_orig.n_unique?

### AutoML

Aim is to understand what is possible with the AutoML libraries. They can aid with the multiple steps required to develop and review ML models. Pre-processing steps are included. Data pipelines can be produced. Model deployment and maintainance can be reviewed.

[AutoML blog post](https://www.baeldung.com/cs/automl)